# EEG Project Group 12

## Obtaining Data and Performing Preprocessing


### Importing Required Libraries

From here upto and including EEG Feature Extraction was taken from the sample example

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
import tensorflow.keras as kr
import seaborn as sns
import os

### Import the Dataset

In [ ]:
%%time
#Get Current Working directory and append the data relative dir
cwd = os.getcwd()
print(cwd)
Salt = cwd + r"/data/salt"
Sour = cwd + r"/data/sour"

#Hold file locations
filesSour=[];
filesSalt=[]; #add more later


#populate file location arrays
for file in os.listdir(Salt):
    if file.endswith('csv'):
        filesSalt.append(os.path.join(Salt,file))
for file in os.listdir(Sour):
    if file.endswith('csv'):
        filesSour.append(os.path.join(Sour,file))
#Test reading files by changing num

num = 5;
Salt_sample = pd.read_csv(filesSalt[num])
Sour_sample =  pd.read_csv(filesSour[num])
print(Salt_sample)
print(Sour_sample)



/Users/ericl/Desktop/4th_Year/MANU 465/465_EEG_Group12
['/Users/ericl/Desktop/4th_Year/MANU 465/465_EEG_Group12/data/sour/6.csv', '/Users/ericl/Desktop/4th_Year/MANU 465/465_EEG_Group12/data/sour/5.csv', '/Users/ericl/Desktop/4th_Year/MANU 465/465_EEG_Group12/data/sour/4.csv', '/Users/ericl/Desktop/4th_Year/MANU 465/465_EEG_Group12/data/sour/1.csv', '/Users/ericl/Desktop/4th_Year/MANU 465/465_EEG_Group12/data/sour/3.csv', '/Users/ericl/Desktop/4th_Year/MANU 465/465_EEG_Group12/data/sour/2.csv']
                     TimeStamp  Delta_TP9  Delta_AF7  Delta_AF8  Delta_TP10  \
0      2024-11-21 02:13:15.650        NaN        NaN        NaN         NaN   
1      2024-11-21 02:13:15.679   1.353651   0.868130    0.00000    0.705829   
2      2024-11-21 02:13:15.680   1.353651   0.868130    0.00000    0.705829   
3      2024-11-21 02:13:15.680   1.353651   0.868130    0.00000    0.705829   
4      2024-11-21 02:13:15.680   1.353651   0.868130    0.00000    0.705829   
...                       

In [3]:
#Mini-Summary of Block
print(f">{len(filesSpicy)} files were added from the Spicy category")
print(f">{len(filesSalty)} files were added from the Salty category")



>0 files were added from the Spicy category
>0 files were added from the Salty category


### Available Features

In [4]:
print("Features generated by the Muse 2 headband:")
pd.DataFrame(sample.columns).T

Features generated by the Muse 2 headband:


NameError: name 'sample' is not defined

### Creating the RAW Dataset

In [ ]:
%%time
#Extract rows 21-25 from all files,
#these are the only 5 features relevant for use in the EEG_feature_extraction function
rowsSpicy=[];
for f in filesSpicy:
    for r in range(pd.read_csv(f).shape[0]):
        rowsSpicy.append(pd.read_csv(f).iloc[r,[0,21,22,23,24,25]])
rowsSalty=[];
for f in filesSalty:
    for r in range(pd.read_csv(f).shape[0]):
        rowsSalty.append(pd.read_csv(f).iloc[r,[0,21,22,23,24,25]])


In [ ]:
#Convert to DataFrames
data_Spicy = pd.DataFrame(rowsSpicy);
original_Spicy = data_Spicy.copy();
data_Spicy

In [ ]:
#Convert to DataFrames
data_Salty = pd.DataFrame(rowsSalty);
original_Salty = data_Salty.copy();
data_Salty

In [ ]:
#quick check of DataFrames
print(f"Spicy data size is: \t{data_Spicy.shape}",f"\nSalty data size is: \t{data_Salty.shape}")

### Remove Empty Rows

In [ ]:
#Remove NaN values
data_Spicy = data_Spicy.dropna()
data_Salty = data_Salty.dropna()

### Convert Datetime Column to Timestamps

In [ ]:
#Required for compatibility with EEG_feature_extraction function
from datetime import datetime

i = 0;
for time in data_Spicy.iloc[:,0]:
    tmstmp = datetime.strptime(str(time),'%Y-%m-%d %H:%M:%S.%f').timestamp()
    data_Spicy.iat[i,0] = (tmstmp);
i=i+1;

i = 0;
for time in data_Salty.iloc[:,0]:
    tmstmp = datetime.strptime(str(time),'%Y-%m-%d %H:%M:%S.%f').timestamp()
    data_Salty.iat[i,0] = (tmstmp);
i=i+1;


In [ ]:
#Quick Check
data_Spicy.head()

### Save Data to File

Alternative STARTING POINT once data collection is finalized. This step was done to bypass having to run the previous section each time which would take a very long time.

In [ ]:
savelocSpicy = cwd + r"\Data\Preprocessed\Spicy.csv"
savelocSalty = cwd + r"\Data\Preprocessed\Salty.csv"

if os.path.exists(savelocSpicy):
    os.remove(savelocSpicy)
if os.path.exists(savelocSalty):
    os.remove(savelocSalty)
data_Spicy.to_csv(savelocSpicy, mode='w', index = False)
data_Salty.to_csv(savelocSalty, mode='w', index = False)




### EEG Feature Generation

In [ ]:
from eegFG import EEG_feature_extraction as FG

#try various combinations of Nsamp and Perio
Nsamp = 50;
Perio = 6;

xSpicy,ySpicy = FG.generate_feature_vectors_from_samples(file_path=savelocSpicy,
                                                         nsamples=Nsamp,
                                                         period=Perio,
                                                         #state=data_Spicy.iloc[:,-1],
                                                         slide_percent=0.05,
                                                         remove_redundant=False,
                                                         cols_to_ignore=None)
xSpicy.shape


In [ ]:
#try various combinations of Nsamp and Perio
Nsamp = 50;
Perio = 5;

xSalty,ySalty = FG.generate_feature_vectors_from_samples(file_path=savelocSalty,
                                                         nsamples=Nsamp,
                                                         period=Perio,
                                                         #state=data_Salty.iloc[:,-1],
                                                         slide_percent=0.06,
                                                         remove_redundant=False,
                                                         cols_to_ignore=None)
xSalty.shape

In [ ]:
#some quick checks
X_Spicy = pd.DataFrame(np.real(xSpicy))
X_Spicy.columns = np.hstack((['Timestamp'],ySpicy))
X_Spicy.describe()

In [ ]:
with pd.option_context('display.max_rows',None,'display.max_columns',None):
    display(pd.DataFrame(pd.DataFrame(X_Spicy).head()))

In [ ]:
X_Salty = pd.DataFrame(np.real(xSalty))
X_Salty.columns = np.hstack((['Timestamp'],ySalty))
X_Salty.describe()

### Analyze the Dataset

### Separate Into Input and Output

### Encoding the Input

### Encoding the Output

### Split into Test and Train

### Feature Scaling

## Random Forest and SVM

### Importing the Models

### Train and Evaluate the Models

## ANN

### Initialize the ANN

### Adding the Input Layer

### Adding the Hidden Layers

### Adding the Output Layer

### Compiling the ANN

### Training the ANN

### Visualization of Validation

### Using the Model to Predict

### Confusion Matrix